In [1]:
from mujoco_py import load_model_from_path, MjSim, MjViewer
from mujoco_py import MjRenderContextOffscreen
import mujoco_py

from IPython.display import HTML, display
from matplotlib import pyplot as plt
from copy import deepcopy
import numpy as np
import imageio
import base64
import types
import glob
import cv2
import io

In [2]:
def reset_mocap_welds(sim):
    """Resets the mocap welds that we use for actuation.
    """
    if sim.model.nmocap > 0 and sim.model.eq_data is not None:
        for i in range(sim.model.eq_data.shape[0]):
            if sim.model.eq_type[i] == mujoco_py.const.EQ_WELD:
                sim.model.eq_data[i, :] = np.array(
                    [0., 0., 0., 1., 0., 0., 0.])
    sim.forward()
    
def save_video(frames, filename='video/mujoco.mp4', fps=60):
    writer = imageio.get_writer(filename, fps=fps)
    for f in frames:
        writer.append_data(f)
    writer.close()
    
def show_video(filname='video/mujoco.mp4'):
    mp4list = glob.glob(filname)
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
        
def show_image(img):
    #cv2.imshow("test", img)
    plt.figure(figsize = (16,9))
    plt.axis('off')
    plt.imshow(img)

In [3]:
model = load_model_from_path('make_urdf/ur5_robotiq.xml')
n_substeps = 1 #20
sim = MjSim(model, nsubsteps=n_substeps)
viewer = MjViewer(sim)

Creating window glfw


In [4]:
arm_joint_list = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
#gripper_joint_list = ['finger_joint','left_inner_knuckle_joint', 'left_inner_finger_joint', 'right_outer_knuckle_joint', \
#              'right_inner_knuckle_joint', 'right_inner_finger_joint']
gripper_joint_list = ['left_finger_joint', 'right_finger_joint']
#gripper_joint_list = ['left_finger_joint']
str = ''
for joint_name in gripper_joint_list:
    str += "{} : {:.3f}, ".format(joint_name, sim.data.get_joint_qpos(joint_name))
print(str)
#print(sim.data.get_joint_qpos('finger_joint'))
#print(sim.data.get_joint_qpos('left_inner_knuckle_joint'))

print(sim.model.joint_names)

left_finger_joint : 0.000, right_finger_joint : 0.000, 
('shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint', 'left_finger_joint', 'left_inner_finger_joint', 'left_inner_knuckle_joint', 'right_finger_joint', 'right_inner_finger_joint', 'right_inner_knuckle_joint', 'free_x_1', 'free_y_1', 'free_z_1')


In [5]:
init_arm_pos = np.array([-np.pi/2, -np.pi/2, -np.pi/2, -np.pi/2, np.pi/2, 0.0])
for joint_idx, joint_name in enumerate(arm_joint_list):
    sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
    sim.data.set_joint_qvel(joint_name, 0.0)
reset_mocap_welds(sim)
    
# Move end effector into position.
gripper_target = np.array([0.0, 0.0, -0.1]) + sim.data.get_body_xpos('wrist_3_link')
gripper_rotation = np.array([1., 0., 1., 0.])
sim.data.set_mocap_pos('robot0:mocap', gripper_target)
sim.data.set_mocap_quat('robot0:mocap', gripper_rotation)
for _ in range(10):
    sim.step()

In [6]:
sim.data.ctrl

array([0., 0.])

In [7]:
lookat_refer = sim.data.get_body_xpos('target_body_1')
viewer.cam.lookat[0] = lookat_refer[0]
viewer.cam.lookat[1] = lookat_refer[1]
viewer.cam.lookat[2] = lookat_refer[2]
viewer.cam.azimuth = -75
viewer.cam.distance = 1.5

In [8]:
gripper_ctrl = -0.005
#init_arm_pos = np.array([-np.pi/2, -np.pi/2, -np.pi/2, -np.pi/2, np.pi/2, 0.0])
#init_gripper_pos = 0.0
#for joint_idx, joint_name in enumerate(arm_joint_list):
#    sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
#    sim.data.set_joint_qvel(joint_name, 0.0)
#for joint_idx, joint_name in enumerate(gripper_joint_list):
#    sim.data.set_joint_qpos(joint_name, init_gripper_pos)
#    sim.data.set_joint_qvel(joint_name, 0.0)
#sim.step()

frames = []
cnt = 0
steps = 300
for j in range(steps):
    sim.data.mocap_pos[0] = sim.data.get_body_xpos('target_body_1') + np.array([0, 0, 0.3])
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    #for joint_idx, joint_name in enumerate(arm_joint_list):
    #    sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
    #    sim.data.set_joint_qvel(joint_name, 0.0)
    #sim.data.set_joint_qpos('finger_joint', min(0.8, init_gripper_pos + 2.0*j/300.0))
    #sim.data.set_joint_qvel('finger_joint', 0.0)
    #sim.data.set_joint_qpos('right_outer_knuckle_joint', min(0.8, init_gripper_pos + 2.0*j/300.0))
    #sim.data.set_joint_qvel('right_outer_knuckle_joint', 0.0)
    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl
        #sim.data.ctrl[gripper_idx] = gripper_ctrl
    #sim.forward()
    #sim.data.ctrl[-1] = -1.0
    sim.step()
    
    #cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

#sim.data.ctrl[-1] = 0.0
        
#save_video(frames, 'video/mujoco.mp4', int(len(frames)/steps*60.0))
save_video(frames, 'video/mujoco.mp4', int(1/viewer._time_per_render))
show_video('video/mujoco.mp4')

In [9]:
for i in range(sim.data.ncon):
    con = sim.data.contact[i]
    print(con.dist, sim.model.geom_id2name(con.geom1), sim.model.geom_id2name(con.geom2))

-0.00010804491878412328 None target_1
-0.00010804491878412328 None target_1
-0.00010804491878412328 None target_1
-0.00010804491878412328 None target_1


In [10]:
gripper_ctrl = -0.005
'''
init_arm_pos = target_qpos
init_gripper_pos = 0.0
for joint_idx, joint_name in enumerate(arm_joint_list):
    sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
    sim.data.set_joint_qvel(joint_name, 0.0)
for joint_idx, joint_name in enumerate(gripper_joint_list):
    sim.data.set_joint_qpos(joint_name, init_gripper_pos)
    sim.data.set_joint_qvel(joint_name, 0.0)
sim.step()
'''
frames = []
cnt = 0
steps = 300
for j in range(steps):
    sim.data.mocap_pos[0] = sim.data.get_body_xpos('target_body_1') + np.array([0, 0, 0.15])
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    #for joint_idx, joint_name in enumerate(arm_joint_list):
    #    sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
    #    sim.data.set_joint_qvel(joint_name, 0.0)
    #sim.data.set_joint_qpos('finger_joint', min(0.8, init_gripper_pos + 2.0*j/300.0))
    #sim.data.set_joint_qpos('finger_joint', 0.0)
    #sim.data.set_joint_qvel('finger_joint', 0.0)

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl
        #sim.data.ctrl[gripper_idx] = gripper_ctrl

    #if j < 10:
    #    sim.data.ctrl[7] = 10.0
    #else:
    #    sim.data.ctrl[7] = 0.0
    #sim.forward()
    sim.step()

    #str = ''
    #for joint_name in gripper_joint_list:
    #    str += "{} : {:.3f}, ".format(joint_name, sim.data.get_joint_qpos(joint_name))
    #print(str)
    
    cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    #cnt += 2*sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)
        
#save_video(frames, 'video/mujoco.mp4', int(len(frames)/steps*60.0))
save_video(frames, 'video/mujoco.mp4', int(1/viewer._time_per_render))
show_video('video/mujoco.mp4')

In [11]:
for i in range(sim.data.ncon):
    con = sim.data.contact[i]
    #print(con.dist, sim.model.body_id2name(sim.model.geom_bodyid[con.geom1]), sim.model.geom_id2name(con.geom2))
    print(con.dist, sim.model.body_id2name(sim.model.geom_bodyid[con.geom1]), sim.model.body_id2name(sim.model.geom_bodyid[con.geom2]))
    

-0.00010764872812029161 box_link target_body_1
-0.00010764872812029161 box_link target_body_1
-0.00010764872812029161 box_link target_body_1
-0.00010764872812029161 box_link target_body_1


In [12]:
gripper_ctrl = 0.005
init_mocap_pos = deepcopy(sim.data.mocap_pos[0])

frames = []
cnt = 0
steps = 300
for j in range(steps):
    sim.data.mocap_pos[0] = init_mocap_pos
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl
        #sim.data.ctrl[gripper_idx] = gripper_ctrl

    sim.step()
    
    cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)
        
#save_video(frames, 'video/mujoco.mp4', int(len(frames)/steps*60.0))
save_video(frames, 'video/mujoco.mp4', int(1/viewer._time_per_render))
show_video('video/mujoco.mp4')

In [13]:
gripper_ctrl = 0.005
init_mocap_pos = deepcopy(sim.data.mocap_pos[0])

frames = []
cnt = 0
steps = 300
for j in range(steps):
    sim.data.mocap_pos[0] = init_mocap_pos + np.array([0, 0, 0.001*j])
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()

    cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    #cnt += 2*sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)
        
save_video(frames, 'video/mujoco.mp4', int(1/viewer._time_per_render))
show_video('video/mujoco.mp4')

In [14]:
import gym.envs.robotics.rotations as tf
from ikpy.chain import Chain

my_chain = Chain.from_urdf_file("make_urdf/ur5.urdf")

print(sim.data.get_body_xpos('wrist_3_link') - sim.data.get_body_xpos('base_link'))

# forward kinematics
arm_pos = sim.data.qpos[:6]
result = my_chain.forward_kinematics([0]+list(arm_pos))
print(result[:3, 3])

# get orientation
print(tf.quat2mat(sim.data.get_body_xquat('wrist_3_link')))
print(result[:3, :3])

#target_quat = sim.data.get_body_xquat('wrist_3_link')
#target_quat = np.array([0.0, 1.0, 0.0, 0.0])
#target_orn = tf.quat2mat(target_quat)
target_orn = tf.euler2mat([-np.pi, 0.0, 0.0])

#target_pos = sim.data.get_body_xpos('target_body_1')
target_pos = np.array([0.0, 0.2, 1.05]) - sim.data.get_body_xpos('base_link')

target = np.eye(4)
target[:3,:3] = target_orn
target[:3,3] = target_pos

print(target)

#target_qpos = my_chain.inverse_kinematics(target_position=target_pos, target_orientation=target_euler, orientation_mode="all")[1:]
#target_qpos = my_chain.inverse_kinematics(target_position=target_pos, target_orientation=target_orn, orientation_mode="all")[1:]
#target_qpos = my_chain.inverse_kinematics(target_position=target_pos)[1:]
arm_pos = sim.data.qpos[:6]
target_qpos = my_chain.inverse_kinematics_frame(target, initial_position=[0.0]+list(arm_pos), \
                                                orientation_mode="all", no_position=False)[1:]
print(arm_pos)
print(my_chain.forward_kinematics([0]+list(target_qpos))[:3,3])
print(target_qpos)

target_qpos = [-1.38786106, -1.97866332, -1.74913957, -0.98458609,  1.57079632,  0.18293526]

[4.52454828e-05 5.99830129e-01 4.53352274e-01]
[4.53117596e-05 5.99829658e-01 4.54351649e-01]
[[ 1.00000000e+00 -2.78896907e-07 -3.05008522e-05]
 [-2.74194141e-07 -9.99999988e-01  1.54184642e-04]
 [-3.05008949e-05 -1.54184633e-04 -9.99999988e-01]]
[[ 1.00000000e+00 -2.82068347e-07 -3.05488177e-05]
 [-2.77347287e-07 -9.99999988e-01  1.54541389e-04]
 [-3.05488609e-05 -1.54541381e-04 -9.99999988e-01]]
[[ 1.0000000e+00 -0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-0.0000000e+00 -1.0000000e+00  1.2246468e-16  6.0000000e-01]
 [ 0.0000000e+00 -1.2246468e-16 -1.0000000e+00  1.8000000e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[-1.38788431 -1.81926163 -1.23162908 -1.66134075  1.57079826  0.1829123 ]
[1.55474532e-08 6.00000003e-01 1.79999988e-01]
[-1.38786106 -1.97866332 -1.74913957 -0.98458608  1.57079634  0.18293527]
